In [2]:
import pandas as pd
import re
import random

In [3]:
df = pd.read_csv('1000sents.csv', nrows=674)

In [4]:
df.columns

Index(['ID', 'HEADWORD', 'POS', 'ENGLISH', 'JAPANESE', 'SPANISH', 'INDONESIAN',
       'EXAMPLE (KO)', 'EXAMPLE (EN)', 'EXAMPLE (JA)', 'EXAMPLE (ES)',
       'EXAMPLE (ID)'],
      dtype='object')

In [5]:
df = df[['EXAMPLE (EN)']]

In [6]:
df

,EXAMPLE (EN)
0,The store opens at 10 A.M.
1,Do you like it? What's the price?
2,My house is near my school.
3,I sometimes drink beer.
4,"Come tomorrow, if possible."
...,...
669,Does your brother wear glasses?
670,I work from 9 a.m. to 6 p.m.
671,Come by this afternoon.
672,How many flights of stairs should I go up?


In [7]:
df.to_csv('sentences.txt', index=False, header=False)

# Create masked sentences for context

In [56]:
def mask_random_word(sentence):
    # Split the sentence into words using regex (considering spaces and punctuations)
#     words = re.findall(r'\b\w+\b', sentence)
    words = re.findall(r'\b(?:[A-Za-z]\w*)\b', sentence)

    if words:
        # Select a random word index
        random_index = random.randint(0, len(words) - 1)

        # Replace the selected word with [MASK]
        words[random_index] = "[MASK]"

        # Join the words back to form the masked sentence
        masked_sentence = " ".join(words)

        return masked_sentence

    return sentence

In [57]:
# Apply the mask_random_word function to the 'EXAMPLE (EN)' column
df['EXAMPLE (EN)'] = df['EXAMPLE (EN)'].apply(mask_random_word)

/tmp/ipykernel_186989/1216340428.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EXAMPLE (EN)'] = df['EXAMPLE (EN)'].apply(mask_random_word)


In [58]:
df

,EXAMPLE (EN)
0,[MASK] store opens at A M
1,Do you like it What [MASK] the price
2,My [MASK] is near my school
3,I sometimes drink [MASK]
4,Come tomorrow [MASK] possible
...,...
669,[MASK] your brother wear glasses
670,I work from a m to p [MASK]
671,Come [MASK] this afternoon
672,How many flights of stairs should I go [MASK]


In [59]:
df.to_csv('masked_sentences.txt', index=False, header=False)

# SQuAD

In [7]:
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

In [1]:
import json

# Load the JSON file
with open('train-v2.0.json', 'r') as json_file:
    data = json.load(json_file)

In [2]:
# data
# Extract 'question' sub-fields and store them in a list
questions = []
for item in data['data']:
    for paragraph in item['paragraphs']:
        for qa in paragraph['qas']:
            questions.append(qa['question'])

In [3]:
len(questions)

130319

In [4]:
questions[0:10]

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?',
 'In what R&B group was she the lead singer?',
 'What album made her a worldwide known artist?',
 "Who managed the Destiny's Child group?",
 'When did Beyoncé rise to fame?',
 "What role did Beyoncé have in Destiny's Child?"]

In [8]:
# Specify the text file path
text_file_path = 'sentences.txt'

# Write the questions list to the text file
with open(text_file_path, 'w') as text_file:
    for question in questions:
        text_file.write(question + '\n')

print(f'Questions have been written to {text_file_path}')

Questions have been written to sentences.txt


### Create masked sentences for context

In [14]:
import re
import random

def mask_random_word(sentence):
    
    words = re.findall(r'\b(?:[A-Za-z]\w*)\b', sentence)

    if words:
        # Select a random word index
        random_index = random.randint(0, len(words) - 1)

        # Replace the selected word with [MASK]
        words[random_index] = "[MASK]"

        # Join the words back to form the masked sentence
        masked_sentence = " ".join(words)

        return masked_sentence

    return sentence

In [15]:
masks = [mask_random_word(question) for question in questions]

In [16]:
masks[0:10]

['[MASK] did Beyonce start becoming popular',
 'What areas did [MASK] compete in when she was growing up',
 'When did Beyonce leave Destiny s Child and [MASK] a solo singer',
 'In what city and state did Beyonce grow [MASK]',
 'In which decade did Beyonce [MASK] famous',
 'In what R B [MASK] was she the lead singer',
 'What album [MASK] her a worldwide known artist',
 'Who managed the Destiny s Child [MASK]',
 '[MASK] did Beyoncé rise to fame',
 'What [MASK] did Beyoncé have in Destiny s Child']

In [18]:
# Specify the text file path
text_file_path = 'masked_sentences.txt'

# Write the questions list to the text file
with open(text_file_path, 'w') as text_file:
    for mask_sent in masks:
        text_file.write(mask_sent + '\n')

print(f'Sentences have been written to {text_file_path}')

Sentences have been written to masked_sentences.txt
